In [2]:
import os
from dotenv import load_dotenv
from supabase import create_client, Client

load_dotenv()
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_SECRET_KEY = os.getenv("SUPABASE_KEY")

In [3]:
url: str = SUPABASE_URL
key: str = SUPABASE_SECRET_KEY

supabase = create_client(url, key)

In [13]:
response = supabase.table("document").select("*").execute()
data = response.data[:1]
data

[{'id': '2922ccf2-2b7c-4109-bbc0-467f6ffd014d',
  'embed_doc': 'III. Hạng mục công việc và người phụ trách\nHạng mục: Xét nhận Đồ án/khóa luận tốt nghiệp Bù tín chỉ\nHạng mục viết tắt: [Xét nhận ĐATN]\nGửi email tới/vào link đặt câu hỏi: huong.nguyenthi1@hust.edu.vn\nMẫu đơn: Tải tại đây ',
  'raw_doc': 'III. Hạng mục công việc và người phụ trách\nHạng mục: Xét nhận Đồ án/khóa luận tốt nghiệp Bù tín chỉ\nHạng mục viết tắt: [Xét nhận ĐATN]\nGửi email tới/vào link đặt câu hỏi: huong.nguyenthi1@hust.edu.vn\nMẫu đơn: Tải tại đây https://husteduvn.sharepoint.com/:w:/s/NguyenXuanTung_PDT/EdDCJnCWLlRHtEafh2p11aoBLB1ev58zMt1tmTuk9x9Nfg?e=1efidj',
  'type_doc': 'Thắc mắc'}]

In [16]:
import asyncio
import json
from openai import OpenAI, AsyncOpenAI
from typing import List, Dict

async def invoke_async(request):    
    client = AsyncOpenAI()
    completion = await client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0,
        messages=[
            {"role": "system", "content": "Bạn là chuyên gia ......"},
            {
                "role": "user",
                "content": request
            }
        ]
    )

    return completion.choices[0].message.content

def create_request(raw_data: str):
    prompt = f"""
    Dựa vào thông tin sau, hãy tạo 3 câu hỏi và câu trả lời liên quan:
    {raw_data}
    
    Format trả về phải là một list các dict với format:
    [
        {{"question": "câu hỏi 1", "answer": "câu trả lời 1"}},
        {{"question": "câu hỏi 2", "answer": "câu trả lời 2"}},
        {{"question": "câu hỏi 3", "answer": "câu trả lời 3"}}
    ]
    """

    return prompt

async def process_documents(data: List[Dict], batch_size: int = 5) -> List[Dict]:
    """Process documents in batches using invoke_async"""
    all_results = []
    
    for i in range(0, len(data), batch_size):
        batch = data[i:i + batch_size]
        batch_results = await process_batch(batch)
        all_results.extend(batch_results)
        print(f"Processed batch {i//batch_size + 1}/{(len(data) + batch_size - 1)//batch_size}")
    
    return all_results

async def process_batch(batch: List[Dict]) -> List[Dict]:
    """Process a single batch of documents"""
    tasks = []
    
    for doc in batch:
        request = create_request(doc['raw_doc'])
        tasks.append(invoke_async(request))
    
    results = await asyncio.gather(*tasks, return_exceptions=True)
    
    processed_data = []
    for doc, result in zip(batch, results):
        if isinstance(result, Exception):
            print(f"Error processing doc {doc['id']}: {str(result)}")
            continue
            
        try:
            # Parse the GPT response into questions and answers
            if "```" in result:
                result = result.replace("```", "").replace("json", "").strip()
            qa_list = json.loads(result)
            for qa in qa_list:
                processed_data.append({
                    "question": qa["question"],
                    "answer": qa["answer"],
                    "raw_data": doc["raw_doc"],
                    "id": doc["id"]
                })
        except Exception as e:
            print(f"Error parsing result for doc {doc['id']}: {str(e)}")
            continue
    
    return processed_data
    

In [17]:
results = await process_documents(data, batch_size=1)
with open('question_generate.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

Processed batch 1/1
